In [27]:
import pandas as pd
import re
import string


### a) Ler o dataset fakeTelegram.BR_2022.csv, o qual está disponível no link a seguir:
### https://drive.google.com/file/d/1c_hLzk85pYw-huHSnFYZM_gndUsYRDm/view?usp=drive_link

In [28]:
fakeTelegram = pd.read_csv("fakeTelegram.BR_2022.csv")

fakeTelegram.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637


### b) Identificar e listar as posições (células) contendo valores faltantes. 


In [18]:
valores_faltantes = (
    fakeTelegram.isna() | 
    fakeTelegram.apply(lambda col: col.astype(str).str.strip() == '', axis=0)
)
valores_faltantes.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,False,False,False,True,True,True,False,False,False,False,False,False,False,True,False,False,False,True,True
1,False,True,False,True,True,True,False,False,False,False,False,False,False,True,False,False,False,True,True
2,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,True,True
3,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False
4,False,False,False,False,False,True,False,False,False,True,False,False,True,True,False,False,False,True,False


Extrair as coordenadas (linha, coluna) das células vazias

In [17]:
coordenadas_vazias = [
    (linha, coluna)
    for linha in valores_faltantes.index[valores_faltantes.any(axis=1)]  # Linhas com vazios
    for coluna in valores_faltantes.columns[valores_faltantes.loc[linha]]  # Colunas vazias na linha
]

print("CÉLULAS FALTANTES")
print(f"Total de células vazias: {len(coordenadas_vazias)}")
print("\nCoordenadas (linha, coluna):")
pd.DataFrame(coordenadas_vazias, columns=["Linha", "Coluna"]).head(10)

CÉLULAS FALTANTES
Total de células vazias: 2544186

Coordenadas (linha, coluna):


,Linha,Coluna
0,0,media
1,0,media_type
2,0,media_url
3,0,score_misinformation
4,0,media_name
5,0,media_md5
6,1,id_member_anonymous
7,1,media
8,1,media_type
9,1,media_url


### c) Contar quantas linhas possuem valores faltantes

In [19]:
linhas_com_dados_vazios = fakeTelegram.isna().any(axis=1) | (fakeTelegram.apply(lambda col: col.astype(str).str.strip() == '', axis=1).any(axis=1))

print(f"Quantidade de linhas com valores faltantes: {linhas_com_dados_vazios.sum()}")

Quantidade de linhas com valores faltantes: 557561


### d) Para cada coluna (feature), contar quantas linhas possuem valores faltantes.

In [21]:
features_faltantes = (
    fakeTelegram.isna().sum() +  # Conta NaNs/None
    fakeTelegram.apply(lambda col: col.astype(str).eq('').sum())  # Conta strings vazias ''
)

print(features_faltantes)

date_message                   0
id_member_anonymous       323341
id_group_anonymous             0
media                     224981
media_type                224981
media_url                 400141
has_media                      0
has_media_url                  0
trava_zap                      0
text_content_anonymous    113385
dataset_info_id                0
date_system                    0
score_sentiment           113429
score_misinformation      390348
id_message                     0
message_type                   0
messenger                      0
media_name                528599
media_md5                 224981
dtype: int64


### e) Identificar e listar as linhas repetidas (duplicadas).

In [8]:
linhas_duplicadas = fakeTelegram[fakeTelegram.duplicated(keep=False)]
quantidade_duplicadas = fakeTelegram.duplicated(keep=False).sum()
print(f"Quantidade de linhas duplicadas: ({quantidade_duplicadas})")
linhas_duplicadas.head()


Quantidade de linhas duplicadas: (0)


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,caracteres


### f) Identificar e listar as posições (células) contendo valores que não pertencem ao domínio (tipo de dados) esperado

In [49]:
def validar_dominio_avancado(df):
    url_regex = re.compile(r'^(?:http(s)?:\/\/)?[\w.-]+(?:\.[\w\.-]+)+[\w\-\._~:/?#[\]@!\$&\'\(\)\*\+,;=.]+$', re.IGNORECASE)
    media_types_validos = ['image/jpg', 'image/jpeg', 'image/png', 'video/mp4', 'video/quicktime', 'url']
    erros = []

    for idx, row in df.iterrows():
        message_type = row['message_type']
        has_media_url = row['has_media_url']
        has_media = row['has_media']
        media = row['media']
        media_type = row['media_type']
        media_url = row['media_url']
        text_content_anonymous = row['text_content_anonymous']
        trava_zap = row['trava_zap']
        score_sentiment = row['score_sentiment']
        score_misinformation = row['score_misinformation']
        media_name = row['media_name']
        media_md5 = row['media_md5']

        # Se tem mídia, precisa do nome e tipo.
        if has_media == True and (pd.isnull(media) or pd.isna(media) or media == '' or pd.isna(media_type) or media_type == '' or pd.isnull(media_type)):
            erros.append((idx, 'has_media', has_media, "has_media = True mas falta nome ou tipo da mídia"))

        # Se tem URL de mídia, ela não pode estar vazia.
        if has_media_url == True and (pd.isna(media_url) or media_url == '' or pd.isnull(media_url)):
            erros.append((idx, 'has_media_url', has_media_url, "has_media_url = True mas a URL está vazia"))

        # Mensagem de texto não pode ter conteúdo vazio.
        if message_type == 'Texto' and (pd.isna(text_content_anonymous) or text_content_anonymous == '' or pd.isnull(text_content_anonymous)):
            erros.append((idx, 'message_type', message_type, "Mensagem de texto sem conteúdo"))

        # Mensagem 'trava zap' também precisa de conteúdo.
        if trava_zap == True and (pd.isna(text_content_anonymous) or text_content_anonymous == '' or pd.isnull(text_content_anonymous)):
            erros.append((idx, 'trava_zap', trava_zap, "Mensagem 'trava zap' sem conteúdo"))

        # Se tem score de sentimento, precisa ter o texto.
        if not pd.isna(score_sentiment) and (pd.isna(text_content_anonymous) or text_content_anonymous == ''):
            erros.append((idx, 'score_sentiment', score_sentiment, "Score de sentimento sem o texto da mensagem"))

        # Se tem score de desinformação, precisa ter o texto.
        if not pd.isna(score_misinformation) and (pd.isna(text_content_anonymous) or text_content_anonymous == ''):
            erros.append((idx, 'score_misinformation', score_misinformation, "Score de desinformação sem o texto da mensagem"))

        # A URL de mídia precisa ser um formato válido.
        if pd.notna(media_url) and media_url != '' and not re.match(url_regex, str(media_url)):
            erros.append((idx, 'media_url', media_url, "URL de mídia em formato inválido"))

        # Se não tem URL de mídia, o campo deve estar vazio.
        if has_media_url == False and pd.notna(media_url) and media_url != '':
            erros.append((idx, 'has_media_url', has_media_url, "Não tem URL de mídia, mas o campo não está vazio"))

        # Se não tem mídia, os campos relacionados devem estar vazios.
        if has_media == False and (pd.notna(media) and media != '' or pd.notna(media_type) and media_type != '' or pd.notna(media_name) and media_name != '' or pd.notna(media_md5) and media_md5 != ''):
            erros.append((idx, 'has_media', has_media, "Não tem mídia, mas há informações nos campos de mídia"))

        # O tipo de mídia precisa ser um dos tipos esperados.
        if has_media == True and pd.notna(media_type) and media_type != '' and media_type not in media_types_validos:
            erros.append((idx, 'media_type', media_type, f"Tipo de mídia '{media_type}' não reconhecido"))

    return erros

In [50]:
erros = validar_dominio_avancado(fakeTelegram)

print("Erros no domínio")
print(f"Total de erros: {len(erros)}")
erros_df = pd.DataFrame(erros, columns=['index', 'coluna', 'valor', 'mensagem'])
erros_df.head()

Erros no domínio
Total de erros: 14161


,index,coluna,valor,mensagem
0,383,media_type,image/mp4,Tipo de mídia 'image/mp4' não reconhecido
1,462,media_type,image/mp4,Tipo de mídia 'image/mp4' não reconhecido
2,754,media_type,image/mp4,Tipo de mídia 'image/mp4' não reconhecido
3,940,media_type,image/mp4,Tipo de mídia 'image/mp4' não reconhecido
4,957,media_type,audio/opus,Tipo de mídia 'audio/opus' não reconhecido


### g) Crie uma coluna chamada “caracteres” contendo a quantidade de caracteres da coluna “text”

In [3]:
fakeTelegram['caracteres'] = fakeTelegram['text_content_anonymous'].str.len()
fakeTelegram[['text_content_anonymous', 'caracteres']].head()

,text_content_anonymous,caracteres
0,Então é Fato Renato o áudio que eu ouvi no wha...,110.0
1,"Saiu no YouTube do presidente a 8 horas atrás,...",141.0
2,"É isso, nossa parte já foi quase toda feita. N...",350.0
3,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,40.0
4,NaN,NaN


### h) Crie uma coluna chamada “words” contendo a quantidade de palavras da coluna “text”.

In [10]:
# Remover pontuação, dividir em palavras e contar
fakeTelegram['words'] = (
    fakeTelegram['text_content_anonymous']
    .str.translate(str.maketrans('', '', string.punctuation))
    .str.split()
    .str.len()
)

# Exibir amostra do resultado
fakeTelegram[['text_content_anonymous', 'words']].head()

,text_content_anonymous,words
0,Então é Fato Renato o áudio que eu ouvi no wha...,19.0
1,"Saiu no YouTube do presidente a 8 horas atrás,...",23.0
2,"É isso, nossa parte já foi quase toda feita. N...",59.0
3,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,7.0
4,NaN,NaN


### i) Crie uma coluna chamada “viral” contendo o valor 0 se o texto da mensagem (valor do campo “text”) não for encontrado em outras linhas e 1, caso contrário. 


In [11]:
# 1 para textos repetidos, 0 para textos unicos
fakeTelegram['viral'] = fakeTelegram['text_content_anonymous'].duplicated(keep=False).astype(int)

# Exibir as primeiras linhas para verificação
fakeTelegram[['text_content_anonymous', 'viral']].head()

,text_content_anonymous,viral
0,Então é Fato Renato o áudio que eu ouvi no wha...,0
1,"Saiu no YouTube do presidente a 8 horas atrás,...",0
2,"É isso, nossa parte já foi quase toda feita. N...",0
3,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,0
4,NaN,1


### j) Crie uma coluna chamada “sharings” contendo a quantidade de vezes que o texto armazenado no atributo “text” aparece no dataset. 


In [12]:
# Etapa 1: Contar quantas vezes cada mensagem aparece
contagem_por_texto = fakeTelegram['text_content_anonymous'].value_counts()

# Etapa 2: Criar a nova coluna 'sharings' com essas contagens
fakeTelegram['sharings'] = fakeTelegram['text_content_anonymous'].map(contagem_por_texto)

# Etapa 3: Garantir que não haja valores nulos e que a coluna seja do tipo inteiro
fakeTelegram['sharings'] = fakeTelegram['sharings'].fillna(0).astype(int)

# Etapa 4: Exibir as primeiras linhas para conferência
print(fakeTelegram[['text_content_anonymous', 'sharings']].head())

,text_content_anonymous,sharings
0,Então é Fato Renato o áudio que eu ouvi no wha...,1
1,"Saiu no YouTube do presidente a 8 horas atrás,...",1
2,"É isso, nossa parte já foi quase toda feita. N...",1
3,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,1
4,NaN,0


### k) Crie uma coluna chamada “sentiment” contendo os valores: -1 para textos negativos, 0 para textos neutros e 1 para textos positivos. 

In [13]:
# Criar e mapear coluna'sentiment' baseada em 'score_sentiment'
fakeTelegram['sentiment'] = fakeTelegram['score_sentiment'].apply(
    lambda x: -1 if x < -0.1 else (1 if x > 0.1 else 0)
)

# Exibir amostra de registros
fakeTelegram[['score_sentiment', 'sentiment']].head()

,score_sentiment,sentiment
0,0.0000,0
1,0.0644,0
2,-0.3551,-1
3,0.0000,0
4,NaN,0


### l) Eliminar as linhas cujo valor da coluna “text” contenham “trava-zaps”. 


In [21]:
# Filtrar e remover linhas onde 'trava-zaps' é True
nao_trava_zap = fakeTelegram[fakeTelegram['trava_zap'] == False]

trava_zap = fakeTelegram[fakeTelegram['trava_zap'] == True]


# Verificar o resultado
print(f"Linhas em que a opção trava zap é false: {len(nao_trava_zap)}")

nao_trava_zap.head()

print(f"Linhas em que a opção trava zap é true {len(trava_zap)}")

Linhas em que a opção trava zap é false: 557570
Linhas em que a opção trava zap é true 16


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,id_message,message_type,messenger,media_name,media_md5,caracteres,words,viral,sharings,sentiment
21944,2022-10-07 07:46:52,NaN,c712c1b704c22bd0cef50bc06125cdbd,NaN,NaN,NaN,False,False,True,NaN,...,53260,Texto,telegram,NaN,NaN,NaN,NaN,1,0,0
89109,2022-10-16 00:45:02,8a30ac374bc4b5930eaf0667a178546a,e56ec342fc599ebb4ed89655eb6f03aa,NaN,NaN,NaN,False,False,True,NaN,...,521324,Texto,telegram,NaN,NaN,NaN,NaN,1,0,0
294541,2022-10-04 14:22:47,39ee10516124280a22f1798f2a41f9a7,959f13e0079883060632c74ffc81c547,NaN,NaN,NaN,False,False,True,NaN,...,27241,Texto,telegram,NaN,NaN,NaN,NaN,1,0,1
324567,2022-10-25 14:55:55,e003fbb6ffedb1838e42360d41cab314,5b10d7739171149be6d9961e3350c071,NaN,NaN,NaN,False,False,True,NaN,...,1182938,Texto,telegram,NaN,NaN,NaN,NaN,1,0,1
389164,2022-10-30 20:19:52,NaN,c8f2de56550ed0bf85249608b7ead93d,NaN,NaN,NaN,False,False,True,NaN,...,28330,Texto,telegram,NaN,NaN,NaN,NaN,1,0,0


### m) Identificar inconsistências entre os atributos (features). 

In [28]:
# Inicializar lista de inconsistências
inconsistencias = []

# 1. has_media = True mas media ou media_type ausentes
filtro1 = (fakeTelegram['has_media'] == True) & (fakeTelegram['media'].isnull() | fakeTelegram['media_type'].isnull())
inconsistencias.append(("has_media=True mas media ou media_type ausentes", fakeTelegram[filtro1]))

# 2. has_media_url = True mas media_url ausente ou vazia
filtro2 = (fakeTelegram['has_media_url'] == True) & (fakeTelegram['media_url'].isnull() | (fakeTelegram['media_url'] == ''))
inconsistencias.append(("has_media_url=True mas media_url ausente ou vazia", fakeTelegram[filtro2]))

# 3. message_type = Texto mas text_content_anonymous ausente ou vazio
filtro3 = (fakeTelegram['message_type'] == 'Texto') & (fakeTelegram['text_content_anonymous'].isnull() | (fakeTelegram['text_content_anonymous'].str.strip() == ''))
inconsistencias.append(("message_type='Texto' mas text_content_anonymous ausente ou vazio", fakeTelegram[filtro3]))

# 4. trava_zap = True mas text_content_anonymous ausente ou vazio
filtro4 = (fakeTelegram['trava_zap'] == True) & (fakeTelegram['text_content_anonymous'].isnull() | (fakeTelegram['text_content_anonymous'].str.strip() == ''))
inconsistencias.append(("trava_zap=True mas text_content_anonymous ausente ou vazio", fakeTelegram[filtro4]))

# 5. score_sentiment preenchido mas text_content_anonymous ausente ou vazio
filtro5 = fakeTelegram['score_sentiment'].notnull() & (fakeTelegram['text_content_anonymous'].isnull() | (fakeTelegram['text_content_anonymous'].str.strip() == ''))
inconsistencias.append(("score_sentiment preenchido mas text_content_anonymous ausente ou vazio", fakeTelegram[filtro5]))

# 6. score_misinformation preenchido mas text_content_anonymous ausente ou vazio
if 'score_misinformation' in fakeTelegram.columns:
    filtro6 = fakeTelegram['score_misinformation'].notnull() & (fakeTelegram['text_content_anonymous'].isnull() | (fakeTelegram['text_content_anonymous'].str.strip() == ''))
    inconsistencias.append(("score_misinformation preenchido mas text_content_anonymous ausente ou vazio", fakeTelegram[filtro6]))

# Exibir os resultados
for descricao, resultado in inconsistencias:
    print(f"\n🔍 {descricao}: {len(resultado)} registros encontrados")
    if not resultado.empty:
        print(resultado.head(3))


🔍 has_media=True mas media ou media_type ausentes: 0 registros encontrados

🔍 has_media_url=True mas media_url ausente ou vazia: 0 registros encontrados

🔍 message_type='Texto' mas text_content_anonymous ausente ou vazio: 15 registros encontrados
               date_message               id_member_anonymous  \
21944   2022-10-07 07:46:52                               NaN   
89109   2022-10-16 00:45:02  8a30ac374bc4b5930eaf0667a178546a   
294541  2022-10-04 14:22:47  39ee10516124280a22f1798f2a41f9a7   

                      id_group_anonymous media media_type media_url  \
21944   c712c1b704c22bd0cef50bc06125cdbd   NaN        NaN       NaN   
89109   e56ec342fc599ebb4ed89655eb6f03aa   NaN        NaN       NaN   
294541  959f13e0079883060632c74ffc81c547   NaN        NaN       NaN   

        has_media  has_media_url  trava_zap text_content_anonymous  ...  \
21944       False          False       True                    NaN  ...   
89109       False          False       True             